In [ ]:
import praw
import pandas as pd
import datetime
import time
import re

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
SCRIPT_ID=""
SCRIPT_SECRET=""
USERNAME=""
PASSWORD=""

In [ ]:
def to_dict(submission_list):
    
    submissions = []
    
    for submission in submission_list:
        entry = {
            "title":[], 
            "score":[], 
            "id":[], "url":[], 
            "comms_num": [],
            "created": [],
            "body":[]
        }
        
        
        entry["title"] = submission.title
        entry["score"] = submission.score
        entry["id"] = submission.id
        entry["url"] = submission.url
        entry["comms_num"] = submission.num_comments
        entry["created"] = datetime.datetime.fromtimestamp(submission.created)
        entry["body"] = submission.selftext
        
        submissions.append(entry)
        
    return submissions

def format_comment(comment):
    
    comment_body = comment.body
    
    comment_body = comment_body.strip()
    
    if (comment_body == "[deleted]") \
        or (comment_body == "[removed]") \
        or re.search(r"All top-level comments must be a story or poem\. Reply here for other comments\.", comment_body):
        return
    
    comment_body = re.sub(r"(“|”)", '"', comment_body)
    comment_body = re.sub(r"\[.*?\]\(.*?\)", '"', comment_body)
    comment_body = re.sub(r"\*\*\*\*\*", "", comment_body)
    comment_body = re.sub(r" > ", "", comment_body)
    comment_body = re.sub(r"(-|_){3,}", "", comment_body)
    
    return comment_body

In [ ]:
reddit = praw.Reddit(client_id=SCRIPT_ID, client_secret=SCRIPT_SECRET, user_agent="writemeastory", username=USERNAME, password=PASSWORD)
writing_prompts = reddit.subreddit('WritingPrompts')
top_writing_prompts = writing_prompts.top()
posts = to_dict(top_writing_prompts)

In [ ]:
with open('test.txt', 'w', encoding="utf-8") as f:
    for post in posts:
        comments = []
        submission = reddit.submission(id=post['id'])
        submission.comments.replace_more(limit=None)
        for comment in submission.comments:
            comments.append(comment)
        print("{} comments found".format(len(comments)))
        formatted_comments = list(filter(None, map(format_comment, comments)))
        for comment in formatted_comments:
            f.write(comment)
        time.sleep(30)
    f.close()